Add intro explanations here....

* setting up dev env and text editor for windows/mac
* os library link
* relative and abspath explanation
* explain importing modules

_Functions called in following code_

* `os.getcwd()`  - Returns the current working directory (usually the directory where the script is located) [More Info](https://www.tutorialspoint.com/python/os_getcwd.htm)
* `os.path.abspath()` - Takes a relative path and returns the respective absolute path (e.g. '~/sample.jpg' becomes '/Users/sampleuser/sample.jpg' on OSX) [More Info](https://www.geeksforgeeks.org/python-os-path-abspath-method-with-example/)
* `os.path.join()` - Takes two string path segments and joins them together, often used to combine folder paths with a filename inside them [More Info](https://www.geeksforgeeks.org/python-os-path-join-method/)
* `os.path.exists()` - Takes a string path and returns true/false whether there is/isn't a file or folder there [More Info](https://www.geeksforgeeks.org/python-check-if-a-file-or-directory-exists/)
* `os.mkdir()` - Takes a string containing desired path and makes a new directory [More Info](https://www.geeksforgeeks.org/python-os-mkdir-method/)

In [ ]:
# import python libraries
import os
import pdf2image

# Getting local directory, probably as an absolute path, but possibly as a relative path
current_directory = os.getcwd()
print('Local root directory: ' + current_directory)

# Converts path to absolute path if it isn't already, otherwise leaves it unchanged
current_directory = os.path.abspath(current_directory)
print('Local root directory (absolute): ' + current_directory)

""" Building path to folder of PDFs using os.path.join() combines current directory with the name of the subdirectory
(pdf) The reason you use os.path.join rather than say, something like current_directory + '/' + 'some_subfolder_name'
is that os.path.join works on multiple operating systems, since Windows uses backslash in filepaths and OSX/Linux
uses forward slash, this is important"""

# combining for full path to folder of pdfs
pdf_directory = os.path.join(current_directory, 'pdf')
print('PDF subfolder Path: ' + pdf_directory)

# repeat path building to non-existant (currently) 'img' and 'txt' output folders
img_directory = os.path.join(current_directory, 'img')
print('Image subfolder Path: ' + img_directory)
txt_directory = os.path.join(current_directory, 'txt')
print('TXT subfolder Path: ' + txt_directory)

# if img directory doesn't exist, make it
if not os.path.exists(img_directory): # this is the same as saying if os.path.exists(img_directory) == False
    os.mkdir(img_directory)
    print('Image subfolder created')
    
# if txt directory doesn't exist, make it
if not os.path.exists(txt_directory):
    os.mkdir(txt_directory)
    print('TXT subfolder created')
    
print('Step completed')